# Sheet for computing the benchmarks and true similarities for the project

In [1]:
# Importing nescessary modules
import os
import re
import shutil
import numpy as np
import pandas as pd


from utils import metafile_handler as mfh
from utils import file_handler as fh

from benchmarks import dtw
from benchmarks import frechet


In [11]:
DATA_PORTO = "../data/chosen_data/porto/"
DATA_ROME = "../data/chosen_data/rome/"
SIM_OUT_FOLDER = "../code/benchmarks/similarities/"

PORTO_FULL_SET = "../data/chosen_data/porto/META-1000.txt"
ROME_FULL_SET = "../data/chosen_data/rome/META-1000.txt"

TEST_SET_PORTO = "../data/chosen_data/porto/META-1000.txt"
TEST_SET_ROME = "../data/chosen_data/rome/META-1000.txt"

PORTO_DTW_FILE = "porto-dtw.csv"
PORTO_FRECHET_FILE = "porto-frechet.csv"
ROME_DTW_FILE = "rome-dtw.csv"
ROME_FRECHET_FILE = "rome-frechet.csv"


In [3]:
def deleteFile(file_name: str) -> None:
    file_path = os.path.join(SIM_OUT_FOLDER, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print("Failed to remove %s. Reason: %s" % (file_path, e))

## DTW distance compuation

Cells for generating and storing the similarities using dtw

In [12]:
## Using Cython DTW, to speed things up

def generate_dtw_similarities(data_folder: str, meta_file: str, file_name: str):
    deleteFile(file_name)

    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    df = dtw.cy_dtw(trajectories)
    df.to_csv(os.path.join(SIM_OUT_FOLDER, file_name))


def generate_frechet_similarities(data_folder: str, meta_file: str, file_name: str):
    deleteFile(file_name)

    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    df = frechet.cy_frechet(trajectories)
    df.to_csv(os.path.join(SIM_OUT_FOLDER, file_name))


def compute_parallell_dtw_similarities(data_folder: str, meta_file: str, file_name: str):
    deleteFile(file_name)

    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    df = dtw.cy_dtw_pool(trajectories)
    df.to_csv(os.path.join(SIM_OUT_FOLDER, file_name))
    


In [ ]:
# DTW over PORTO
generate_dtw_similarities(DATA_PORTO, TEST_SET_PORTO, PORTO_DTW_FILE)

In [39]:
# DTW over ROME
generate_dtw_similarities(DATA_ROME, TEST_SET_ROME, ROME_DTW_FILE)

In [40]:
# Frechet over Porto
generate_frechet_similarities(DATA_PORTO, TEST_SET_PORTO, PORTO_FRECHET_FILE)

In [ ]:
# Frechet over rome
generate_frechet_similarities(DATA_ROME, TEST_SET_ROME, ROME_FRECHET_FILE)

In [10]:
# Test parallell Porto
compute_parallell_dtw_similarities(DATA_PORTO, TEST_SET_PORTO, PORTO_DTW_FILE)


          P_AAA     P_AAB     P_AAC     P_AAD     P_AAE     P_AAF     P_AAG  \
P_AAA  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
P_AAB  0.876957  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
P_AAC  0.403381  1.097134  0.000000  0.000000  0.000000  0.000000  0.000000   
P_AAD  0.439374  0.477311  0.725498  0.000000  0.000000  0.000000  0.000000   
P_AAE  0.786296  1.138453  0.904284  0.853663  0.000000  0.000000  0.000000   
...         ...       ...       ...       ...       ...       ...       ...   
P_BMH  0.858050  0.130539  1.107160  0.371797  1.066198  0.783858  1.387092   
P_BMI  0.664774  1.005566  0.631517  0.685607  0.975457  0.916853  1.288154   
P_BMJ  1.541740  1.001138  1.435010  1.084834  1.672779  0.780296  0.689485   
P_BMK  0.980992  0.658459  1.148353  0.335771  1.093989  0.849177  1.173449   
P_BML  0.999934  0.654211  1.228376  0.825437  0.953296  0.379451  1.300827   

          P_AAH     P_AAI     P_AAJ  ...     P_BMC 

In [13]:
# Test parallell Rome
compute_parallell_dtw_similarities(DATA_ROME, TEST_SET_ROME, ROME_DTW_FILE)

In [17]:
## Python DTW

files = mfh.read_meta_file(TEST_SET_PORTO)
trajectories = fh.load_trajectory_files(files, DATA_PORTO)

#df = dtw.py_dtw(trajectories)



## Frechet distance computation

In [ ]:
## Python frechet

files = mfh.read_meta_file(TEST_SET)
trajectories = fh.load_trajectory_files(files, DATA_PORTO)

df = frechet.py_frechet(trajectories)


In [ ]:
## Cython frechet 

files = mfh.read_meta_file(TEST_SET)
trajectories = fh.load_trajectory_files(files, DATA_PORTO)

df = frechet.cy_frechet(trajectories)